# Config-2: Computing differentiable features & collision evaluation

The [1st tutorial](https://marctoussaint.github.io/robotics-course/tutorials/1a-configurations.html) introduced the basic concepts of features.
The [lecture script section 'Kinematics'](https://marctoussaint.github.io/robotics-course/script/script.html#kinematics) introduces this a bit more formally.

This tutorial first gives an overview over available features, and then provides more details specifically on collision features.

## Recap of evaluating features directly
Let's first recap how features are directly computed on a configuration - as introduced in the [1st tutorial](config_1_intro.ipynb):

In [ ]:
import robotic as ry

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('panda/panda.g'))
q = C.getJointState()

In [ ]:
[y,J] = C.eval(ry.FS.position, ['gripper'])
print('feature value:', y, '\nJacobian:', J)

In [ ]:
# the x-axis of the given frame in world coordinates
C.eval(ry.FS.vectorX, ['gripper'])

The signature of the `eval` method is

* The feature symbol (`FS.<name>` in python; `FS_<name>` in cpp)
* The set of frames it refers to, given as list of frame names
* Optionally: A scale, that can also be a matrix to down-project a feature (see below)
* Optionally: A target, which changes the zero-point of the features (see below)

## List of features
Here is a full list of feature symbols:

In [ ]:
ry.FS.__members__.keys()

But some of these (esp later ones) are exotic or experimental. The core pre-defined features are the following:

| FS | frames | D | description |
|:---|:---:|:---:|:---|
| position | [A] | 3 | 3D position of A in world coordinates |
| positionDiff | [A,B] | 3 | difference of 3D positions of A and B in world coordinates |
| positionRel | [A,B] | 3 | 3D position of A in B-coordinates |
| quaternion | [A] | 4 | 4D quaternion of A in world coordinates\footnote[There is ways to handle the invariance w.r.t.\ quaternion sign properly.] |
| quaternionDiff | [A,B] | 4 | ... |
| quaternionRel | [A,B] | 4 | ... |
| pose | [A] | 7 | 7D pose of A in world coordinates |
| poseDiff | [A,B] | 7 | ... |
| poseRel | [A,B] | 7 | ... |
| vectorX | [A] | 3 | The x-basis-vector of frame A in world coordinates |
| vectorXDiff | [A,B] | 3 | The difference of the above for two frames A and B |
| vectorXRel | [A,B] | 3 | The x-basis-vector of frame A in B-coordinates |
| vectorY... | | | same as above |
| scalarProductXX | [A,B] | 1 | The scalar product of the x-basis-vector of frame A with the x-basis-vector of frame B |
| scalarProduct... | [A,B] | | as above |
| angularVel | [A] | 3 | The angular velocity of frame A across two configurations (must be order=1!) |
| accumulatedCollisions | [] | 1 | The sum of collision penetrations; when negative/zero, nothing is colliding |
| jointLimits | [] | 1 | The sum of joint limit penetrations; when negative/zero, all joint limits are ok |
| negDistance | [A,B] | 1 | The NEGATIVE distance between convex meshes A and B, positive for penetration |
| qItself | [] | n | The configuration joint vector |
| aboveBox | [A,B] | 4 | when all negative, A is above (inside support of) the box B |
| insideBox | [A,B] | 6 | when all negative, A is inside the box B |

## Position features
Let's briefly clarify the difference between `position`, `positionDiff`, and `positionRel`:

* `position` is the position of a frame `A` in world coordinates
* `positionDiff` is the difference (in world coordinates) of position `B` MINUS position `A`
* `positionRel` is the position of frame `A` in the RELATIVE coordinates of frame `B`

The best example is if `A` is a camera: Assume you would like to frame `B` to be positioned exactly at coordinate (0,0,-.3) in the coordinate frame of `A` -- meaning exactly 30cm centrally in front of the camera -- then the following feature would evaluate the error:

In [ ]:
C.eval(ry.FS.positionRel, ['gripper', 'panda_joint1'], scale=[1], target=[0, 0, .3])

Concretely, if you minimize the above feature, the `gripper` will look exactly(=centrally) at `panda_joint1` with 30cm distance.

## Scalar product features
The `scalarProduct` features are also very useful to define relative between frames (e.g. for grasping). For instance, `(FS.scalarProductXX, ['handL', 'handR'], target=[1])` says that the scalar product of the x-axes (e.g. directions of the index finger) of both hands should equal 1, which means they are aligned. And
```
(FS.scalarProductXY, ['handL', 'handR'])
(FS.scalarProductXZ, ['handL', 'handR'])
```
says that the the x-axis of handL should be orthogonal (zero scalar product) to the y- and z-axis of handR. So this also describes aligning both x-axes. However, this formulation is much more robust, as it has good error gradients around the optimum.

## Scale and target transformation
Let's explain the `scale` and `target` in detail: Formally, specifying a target and scale redefines a feature to become
$$
  \phi(x) \gets \texttt{scale} \cdot (\phi(x) - \texttt{target})
$$
The target needs to be a $D$-dim vector and defines the zero-point of the feature.

The scale can be

* a scalar (just a factor),
* a $D$-vector (multiplying element-wise to the feature)
* or a **matrix**.

We can do interesting things when `scale` is a matrix. For instance, if we only want the $xy$-position of a frame returned, we can choose a matrix $S=\begin{pmatrix}1 & 0 & 0 \\ 0 & 1 & 0\end{pmatrix}$, which multiplies to the 3D feature to return a 2D feature. In code:

In [ ]:
C.eval(ry.FS.position, ['gripper'], [[1,0,0],[0,1,0]])

A very useful application is again if we care about a camera looking at a point: If we want frame `B` to appear centrally in the $xy$-plane of frame `A`, we can mimimize the feature:

In [ ]:
C.eval(ry.FS.positionRel, ['gripper', 'panda_joint1'], [[1,0,0],[0,1,0]])

## Collision features

Let's evaluate the accumulative collision scalar and its Jacobian

In [ ]:
C.computeCollisions() #collisions/proxies are not automatically computed on set...State
C.eval(ry.FS.accumulatedCollisions, [])

Let's move into collision and redo this

In [ ]:
import robotic as ry
C = ry.Config()
C.clear()
C.addFile(ry.raiPath('panda/panda.g'))
C.addFile(ry.raiPath('panda/panda.g'), 'r_')
base_r = C.getFrame('r_panda_base')
base_r.setPosition([.0, .5, .0])

In [ ]:
C.selectJoints(['panda_joint1', 'panda_joint2', 'r_panda_joint1', 'r_panda_joint2'])
C.setJointState([1.,-.8,-1.,-.8])
C.view()

The configuration is now in collision. We can evaluate between some specific shape-pairs that show this:

In [ ]:
C.eval(ry.FS.negDistance, ['palm', 'r_palm'])

In [ ]:
C.eval(ry.FS.negDistance, ['panda_coll7', 'r_panda_coll7'])

In [ ]:
C.eval(ry.FS.negDistance, ['panda_coll6', 'r_panda_coll6'])

However, the above features only return penetration (=negDistance) between specific pairs of shapes. For holistic collision checking we query if any pair of shapes collides. This is called **broad phase collision checking**. The following does this (calling `fcl` internally):

In [ ]:
C.computeCollisions()
C.getCollisions()

In [ ]:
C.getCollisionsTotalPenetration()

The last command is useful for what is classically called binary collision check: if `totalPenetration` is zero, we have no collisions.

Finally, we can get the same information in a differentiable manner, as a feature:

In [ ]:
C.eval(ry.FS.accumulatedCollisions, [])

## Enabling/disabling collisions

In a typical robotic configuration we might have both, visual and (more coarse) collision shapes. Broadphase collision checking should only include the collision shapes. Further, after broadphase collision checking one typically wants to filter out some collisions: shapes of consecutive robot links should usually not be included in collision checking.

In the rai code, shapes have an *integer* `contact` parameter to control this, with the following semantics:

* contact = 0 (default) --> the shape never collides (e.g., is a visual)
* contact = 1 --> the shape collides with all other shapes
* contact < -k (some negative number) --> the shape collides with all shapes except for those that are k-th order parents

So, the latter setting is very natural in robot chains: contact=-1 means "not colliding with the parent link". To be precise, the "k-th order parenthood" means frames between which there are k joints or less.

Note `contact` needs to be set *before* the first broadphase collision checking, as it is used in the construction of the underlying collision engine.

In [ ]:
del C